# Quotient Rings and Field Extensions

**Module 02** | 02-rings-fields-polynomials

*quotient(), building GF(4) from F_2[x]*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **quotient rings and field extensions**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Polynomial Division and Irreducibility](02e-polynomial-division-irreducibility.ipynb).
- Concepts and notation introduced in the previous notebook.

## Quotient Ring Construction

In [ ]:
# Build GF(4) as F_2[x] / (x^2 + x + 1)
R.<x> = PolynomialRing(GF(2))
S.<a> = R.quotient(x^2 + x + 1)
print(list(S))

## Arithmetic in the Quotient

In [ ]:
# Multiply elements in GF(4)
R.<x> = PolynomialRing(GF(2))
S.<a> = R.quotient(x^2 + x + 1)
print(f'a * (a+1) = {a * (a+1)}')

## Verifying Field Properties

In [ ]:
# Check that our quotient ring is actually a field
R.<x> = PolynomialRing(GF(2))
S.<a> = R.quotient(x^2 + x + 1)
print('Is field?', S.is_field())

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **quotient rings and field extensions**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*